# LELA70331 Computational Linguistics Week 7

This week we are going to take a look at chatbots

## Rule-based chatbot: Eliza

As described in this week's lecture, Eliza is a chatbot that simulates a Rogerian therapist, making use of a set of rules in the form of regular expressions. At the heart of Eliza are the substitution function (re.sub in Python) and grouping. We'll start with a quick recap as to what these are.

We need to import the Regular Expressions module in Python.

In [ ]:
import re

This gives as access to the very useful function re.sub.

### re.sub()

This finds all occurences of a given sequence and replaces it with a sequence provided:

In [ ]:
utt = 'walked'
re.sub('ed','ing',utt)

### Groups

Grouping is a very powerful technique for picking out substrings from a string that matches a specified pattern. Parentheses are used to indicate the start and end of the substring. It is very powerful when combined with substitution.

You can use parentheses to capture a particular substring within a pattern and then use it in your replacement string within sub. For example:


In [ ]:
utt = "procrastinating"
re.sub('([a-z]+)ing','\\1ed',utt)

## A very simple Elizabot

The code below implements a very simple Eliza. The function respond takes an utterance as input and using re.sub to generate responses. The loop below the function creates a simple interface that takes user input and prints the response.

We can extend Eliza's ability by adding additional rules.

In [ ]:
def respond(utt):
  utt = re.sub('hello my name is (.+)','hello \\1 how are you feeling today', utt)
  return utt

In [ ]:
utt = ""
while utt != 'goodbye':
    utt = input('> ')
    reply = respond(utt)
    if reply != utt:
        print(reply)
    else:
        if utt != "goodbye":
            print("Can you rephrase that?")

### Activity

Add patterns (using substitutions and grouping) to the respond function that will allow Eliza to conduct both of these conversations. Test your system by conducting the conversation with Eliza.

User: hello my name is emma <br>
Eliza: Hello emma my name is Eliza. How are you feeling today? <br>
User: i am feeling very happy <br>
Eliza: Do you often feel happy? <br>
User: yes since I started my new job <br>
Eliza: Can you tell me about starting your new job? <br>

User: hello my name is john <br>
Eliza: Hello john, my name is Eliza. How are you feeling today? <br>
User: i am feeling pretty happy <br>
Eliza: Do you often feel happy? <br>
User: yes since I moved house <br>
Eliza: Can you tell me about moving house? <br>

### Reverse engineering the NLTK Eliza

There have been many implementations of Eliza over the years. One version is built into the NLTK toolkit. This can be run as follows:


In [ ]:
import nltk

In [ ]:
nltk.chat.eliza.demo()

Activity: Conduct a four-turn-each conversation of your own with the NLTK Eliza. Adds the substitution that you think Eliza is using to generate the responses to your own chatbot using the code below. Where you find Eliza's response to be lacking, update the substitution to give a better response.

In [ ]:
def respond(utt):
  utt = re.sub('hello my name is (.+)','hello \\1 how are you feeling today', utt)
  return utt

In [ ]:
utt = ""
while utt != 'goodbye':
    utt = input('> ')
    reply = respond(utt)
    if reply != utt:
        print(reply)
    else:
        if utt != "goodbye":
            print("Can you rephrase that?")

### Text Generation

We are now going to look at methods for creative generation of text

### Ngram-based generation
The simplest way to perform generation is to learn and apply n-gram probabilities. The way this works is that we first calculate the probability of each word in a corpus following each other word. We then pick a random word to start the sentence and the select a next word that is probable given that word. We then output that next word and then select another word that is probable given THAT word. We then repeat for N words.

This allows us to generate sentences that look a little like natural language, and to imitate particular genres and styles.

In [ ]:
import nltk	
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

In [ ]:
kjv = nltk.corpus.gutenberg.words('bible-kjv.txt')
macbeth = nltk.corpus.gutenberg.words('shakespeare-macbeth.txt')
bigrams = nltk.bigrams(kjv)
cfd = nltk.ConditionalFreqDist(bigrams)

In [ ]:
cfd['the']

In [ ]:
def generate_argmax(cfdist, word, num=15):
    for i in range(num):
        print(word, end=' ')
        word = cfdist[word].max()
import random
def generate_random(cfdist, word, num=15):
    for i in range(num):
        print(word, end=' ')
        word = random.choices(list(cfd[word].keys()), weights=cfd[word].values(), k=1)[0]

In [ ]:
generate_argmax(cfd,"the")

### Text generation with neural language models
The example above uses a very simple language model. However much better performance can be achieved using the same basic principle but employing neural network-based language models and huge amounts of data. We are going to use a pretrained model called GPT-2 (https://openai.com/blog/better-language-models/) which has been made publically available and is very widely used. It is a neural network model using an architecture known as a transformer. 

In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, set_seed
import random

In [ ]:
model_name = 'gpt2-medium'
num_samples = 1
max_length = 100
top_k = 10 

In [ ]:
set_seed(42)
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

In [ ]:
prompts = []
prompt = input()
prompts.append(prompt)

In [ ]:
output = {}
for text in prompts:
	input_ids = tokenizer.encode(text, return_tensors='pt')
	max_length = len(text.split()) + max_length
	responses = []
	responses = model.generate(input_ids, max_length = max_length, do_sample=True, top_k=top_k, num_return_sequences=num_samples)

	responses = responses[:, input_ids.shape[-1]:]
	output[text] = []
	for i, r in enumerate(responses):
		response = tokenizer.decode(r, skip_special_tokens=True)
		output[text].append(response)

In [ ]:
for k in output:
    print('prompt: ' + k + '\n')
    for i, r in enumerate(output[k]):
        print('================================== Output ==================================')
        print(k + " " + r.strip() + '\n')
    print('********************************************************************************************************************')
    print('')

## Corpus-based chatbots

Training and running a corpus-based chatbot takes more steps than we have time for today. If you want to have a go in your own time, then you will find a tutorial for doing so in Pytorch here:

https://pytorch.org/tutorials/beginner/chatbot_tutorial.html

There is a link at the top of the page to open the notebook in Colab.

To run this in colab easily just put the following code block at the top and run it before working through the rest of the notebook.

In [ ]:
! wget https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
! unzip movie-corpus.zip
! mkdir data
! mv movie-corpus data/